origin: http://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html  
translator: Hongpu Liu

In [1]:
%matplotlib inline

# 用PyTorch实现深度学习系统
## 1. 深度学习系统的构造单元：仿射映射，非线性和目标函数
在深度学习系统是由线性单元与非线性单元以某种明智的方式组合在一起的。通过引入非线性单元，我们可以打造更加强力的模型。在本节中，我们将使用这些核心部件，然后构造目标函数，最后将看到模型是如何训练的。

### 1.1 仿射映射
深度学习第一个核心的组件就是仿射映射，仿射映射是一个函数$f(x)$：
$$f(x)=Ax+b$$
其中$A$是矩阵，而$x,b$是向量。其中$A$和$b$是可学习的参数。通常$b$表示*bias*项。

Pytorch和大部分深度学习框架与线性代数稍有不同，输入被映射到行而不是列。也就是说输出的第i行，是输入的第i行经过矩阵$A$的变换再加上偏差项得到的。可以看一下下面的例子：

In [2]:
from __future__ import print_function, division

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
lin = nn.Linear(5, 3) # 该变换将R^5映射到R^3，其参数是A和b

# 数据的维度是2x5， 而映射是从5维到3维……能够用A映射数据？
data = autograd.Variable(torch.randn(2, 5))
print(lin(data)) # 可以映射

Variable containing:
 0.1755 -0.3268 -0.5069
-0.6602  0.2260  0.1089
[torch.FloatTensor of size 2x3]



### 1.2 非线性单元
首先，要注意到以下的事实，它将解释为何需要一个非线性单元。假设我们有两个仿射映射$f(x)=Ax+b$和$g(x)=Cx+d$，那么$f(g(x))$将会是：

$$f(g(x))=A(Cx+d)+b=ACx+(Ad+b)$$

$AC$是一个矩阵，$Ad+b$是向量。我们可以看到两个仿射映射的组合依旧是一个仿射映射。

通过这一事实，我们可以发现：如果神经网络由非常多的仿射运算直接组合，那么模型的能力与一个仿射映射并没有什么不同。

但是，如果我们在仿射映射之间引入了非线性单元，前面的情况就不会发生。我们从而可以构建一个更为强力的模型。

有一些核心的非线性函数非常常用：$tanh(x)$，$\sigma(x)$和$ReLU(x)$。你可能会奇怪：“为什么是这些函数，我可以想到大量其他的非线性函数。”之所以使用这些函数是因为它们的梯度已于计算，而对于学习问题梯度的计算是非常重要的。例如：

$$\frac {d\sigma} {dx} = \sigma(x)(1-\sigma(x)$$

**注意：**尽管你可能在AI基础课程中遇到的神经网络会使用$\sigma(x)$作为默认非线性单元。但是在实践中人们会避免这样做。这是由于随着参数绝对值的增长，会产生**梯度消失**。小的梯度意味着难以学习。大部分人会使用**tanh**和**ReLU**。

In [4]:
# 在PyTorch中，大部分的非线性单元都在torch.nn.functional中（我们以将将其import为F）
# 注意：非线性单元并不像仿射映射那样包含参数。
# 也就是说它们没有可以用来更新和训练的权重。
data = autograd.Variable(torch.randn(2, 2))
print(data)
print(F.relu(data))

Variable containing:
-0.5404 -2.2102
 2.1130 -0.0040
[torch.FloatTensor of size 2x2]

Variable containing:
 0.0000  0.0000
 2.1130  0.0000
[torch.FloatTensor of size 2x2]



### 1.3 Softmax与概率
$Softmax(x)$也是一个非线性单元，该单元通常用于网络的最后一步操作。这是因为该运算可以把一个实数值向量转变为一个概率分布。它的定义如下：令$x$是一个实值向量（可以为正、可以为负，没有任何约束）。然后$Softmax(x)$函数输出的第i个元素为：

$$\frac {exp(x_i)} {\Sigma_jexp(x_j)}$$

要注意该函数的输出是一个概率分布：每个元素都是非负的，并且和等于1。

也可以把它理解为对输入向量应用逐元素的指数运算，来保证输出值非负，然后再除以归一化常量。

In [5]:
# Softmax也在torch.nn.functional中
data = autograd.Variable(torch.randn(5))
print(data)
print(F.softmax(data, dim=0))
print(F.softmax(data, dim=0).sum()) # 由于输出是一个分布，因此和为1
print(F.log_softmax(data, dim=0)) # Pytorch还支持对数Softmax

Variable containing:
 1.3800
-1.3505
 0.3455
 0.5046
 1.8213
[torch.FloatTensor of size 5]

Variable containing:
 0.2948
 0.0192
 0.1048
 0.1228
 0.4584
[torch.FloatTensor of size 5]

Variable containing:
 1
[torch.FloatTensor of size 1]

Variable containing:
-1.2214
-3.9519
-2.2560
-2.0969
-0.7801
[torch.FloatTensor of size 5]



### 1.4 目标函数
神经网络训练的目的是最小化某个函数，该函数被称为目标函数（也经常被称为loss函数和cost函数）。网络训练的过程为：首先选择一个训练样本，将该样本传递给神经网络，然后计算网络输出的损失。然后用损失函数关于权重的导数来更新模型的参数。直觉上说，如果神经网络非常确信自己的答案，而该答案是错的，那么损失就会很高。而如果神经网络非常信任的答案是正确的，损失就会很低。

最小化训练样本的损失函数的背后思想是，获得理想的泛化，对来自开发集、测试集甚至生产环境中从未见过的样本具有较小的损失。常见的一个损失函数是**负对数似然损失**，该损失函数在多分类问题中经常被用到。在监督多分类问题中，该函数意味着最小化产生正确输出的负对数概率（等价来说就是最大化产生正确输出的对数概率）。

## 2. 优化与训练
之前我们看到**autograd.Variable**保留了用来计算梯度的所有信息。由于我们的loss是一个**autograd.Variable**，因此我们可以计算loss关于所有参数的梯度。然后利用该梯度执行参数更新。令$\theta$为参数，$L(\theta)$是损失函数，$\eta$是正的学习率，那么采用如下更新：

$$\theta^{(t+1)}=\theta^{(t)}-\eta\nabla_\theta L(\theta)$$

有大量的算法和研究试图改善原始的梯度更新算法。许多算法试图在训练时使用变化的权重。除非你对这些算法感兴趣，无需担忧这些算法是如何设计的。PyTorch提供了许多这样的算法，这些算法都封装在**torch.optim**中。使用最简单的梯度更新和复杂的更新算法，从接口上几乎是一样的。尝试不同的更新算法以及参数，对于改善网络的性能是至关重要的。通常为了提高性能，会用**Adam**或**RMSProp**来代替原始的随机梯度下降算法。

## 3. 在PyTorch中创建网络组件
在我们把注意力移到NLP之前，我们首先用PyTorch构建一个神经网络作为例子。该网络只包含仿射映射与非线性激活单元，采用内建的负对数似然作为损失函数，然后用反向传播更新参数。

所有的神经网络组件都继承自**nn.Module**，并重载了**forward()**方法。**nn.Module**为你的组件提供了许多功能。例如，对可训练参数保持跟踪，通过**.cpu()**和**.cuda()**函数在CCPU和GPU之间交换数据。

下面编写一个用来注解神经网络的例子，该网络的输入是一个稀疏的词袋表示，输出是两个标签“English”和“Spanish”的概率分布。该模式只是一个逻辑斯蒂回归。

### 3.1 例子：逻辑斯蒂回归词袋分类器
我们的模型将把词袋表示映射为标签的对数概率。我们为每个词在词典中指定一个索引。例如，我们的词汇表只包含“hello”和“world”两个词，它们的索引分别是0和1。则句子“hello hello hello hello”的词袋表示为：
$$[4,0]$$
而“hello world world hello”的词袋表示为：
$$[2,2]$$
词袋表示的通用形式为：
$$[Count(hello),Count(world)]$$
将词袋向量记为$x$，则网络的输出为：
$$logSoftmax(Ax+b)$$
我们把输入传递给仿射映射，然后计算其对数softmax。

In [6]:
data = [
    ("me gusta comer en la cafeteria".split(), "SPANISH"),
    ("Give it to me".split(), "ENGLISH"),
    ("No creo que sea una buena idea".split(), "SPANISH"),
    ("No it is not a good idea to get lost at sea".split(), "ENGLISH")   
]

test_data = [
    ("Yo creo que si".split(), "SPANISH"),
    ("it is lost on me".split(), "ENGLISH")
]

# word_to_ix 把每个词映射为词汇表中的唯一整数，该整数作为词袋向量的索引
word_to_ix = {}
for sent, _ in data + test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

class BoWClassifier(nn.Module): # 继承自nn.Module
    
    def __init__(self, num_labels, vocab_size):
        # 首先调用nn.Module的init方法，无需对语法格式疑惑，记住就好
        super(BoWClassifier, self).__init__()
        
        # 定义需要的参数。在本例中我们需要 A 和 b，
        # 即仿射映射所需的参数。
        # PyTorch内置了 nn.Linear() ，提供了仿射映射。
        # 确保你已经明白了为什么输入维度是 vacab_size ，而输出维度是 num_labels！
        self.linear = nn.Linear(vocab_size, num_labels)
        
        # 注意：非线性单元 log softmax 没有可训练的参数。
    
    def forward(self, bow_vec):
        # 把输入传入 linear 层， 然后在传给 log_softmax 层。
        # nn.functional 中包含了大量的非线性单元和其他函数
        return F.log_softmax(self.linear(bow_vec), dim=1)

def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

# 模型存储了它的参数，在本例中是 A 和 b 。
# 在本例中就是 self.linear = nn.Linear(...) ， 模型保存了关于Linear参数的所有信息
# 在本例中第一个输出的是 A ， 第二个是 b 。
for param in model.parameters():
    print(param)
    
# 要运行模型，需要为模型传递一个词袋向量，并将该向量封装到一个 autograd.Variable 中
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)
log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

{'en': 3, 'No': 9, 'buena': 14, 'it': 7, 'at': 22, 'sea': 12, 'cafeteria': 5, 'Yo': 23, 'la': 4, 'to': 8, 'creo': 10, 'is': 16, 'a': 18, 'good': 19, 'get': 20, 'idea': 15, 'que': 11, 'not': 17, 'me': 0, 'on': 25, 'gusta': 1, 'lost': 21, 'Give': 6, 'una': 13, 'si': 24, 'comer': 2}
Parameter containing:

Columns 0 to 9 
 0.1194  0.0609 -0.1268  0.1274  0.1191  0.1739 -0.1099 -0.0323 -0.0038  0.0286
 0.1152 -0.1136 -0.1743  0.1427 -0.0291  0.1103  0.0630 -0.1471  0.0394  0.0471

Columns 10 to 19 
-0.1488 -0.1392  0.1067 -0.0460  0.0958  0.0112  0.0644  0.0431  0.0713  0.0972
-0.1313 -0.0931  0.0669  0.0351 -0.0834 -0.0594  0.1796 -0.0363  0.1106  0.0849

Columns 20 to 25 
-0.1816  0.0987 -0.1379 -0.1480  0.0119 -0.0334
-0.1268 -0.1668  0.1882  0.0102  0.1344  0.0406
[torch.FloatTensor of size 2x26]

Parameter containing:
 0.0631
 0.1465
[torch.FloatTensor of size 2]

Variable containing:
-0.5378 -0.8771
[torch.FloatTensor of size 1x2]



上面例子中最后的输出对应的是**ENGLISH**和**SPANISH**两个标签的对数概率。我们从未定义过标签，因此需要在训练之前对其进行定义。

In [7]:
label_to_ix = {"SPANISH": 0, "ENGLISH": 1}

接下来我们来训练网络。我们需要把样本送入模型来计算对数概率，然后计算损失函数，接下来计算损失函数关于参数的梯度，最后用梯度对参数进行更新。损失函数使用PyTorch内置的**nn.NLLLoss()**（负对数似然损失）。同样，在**torch.optim**中定义了优化算法，我们使用**SGD**（随机梯度下降）算法。

注意：NLLLoss的输入是一个对数概率向量和一个目标标签。它不会为我们计算对数概率，因此网络的最后一层是**log softmax**。损失函数**nn.CrossEntropyLoss()**与**NLLLoss**功能相同，唯一的差别是：它会帮助我们计算对数softmax。

In [8]:
# 在训练之前，运行一遍测试数据。仅仅为了和训练之后进行对比。
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)
    
# 打印与“creo”对应的参数矩阵列
print(next(model.parameters())[:, word_to_ix["creo"]])

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# 通常需要把训练数据进行多次迭代，迭代100次对于真实数据集来说确实太大。
# 通常迭代30轮是比较合理的。
for epoch in range(100):
    for instance, label in data:
        # 第一步：由于Pytorch会累加梯度，因此首先将梯度清零
        model.zero_grad()
        
        # 第二步：生成词袋向量和标签，并封装到 Variable 中。
        # 例如 SPANISH 封装为 0 .这样损失函数就知道对数概率的第 0 个元素
        # 是 SPANISH 对应的对数概率。
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))
        
        # 第三步：运行前馈传播
        log_probs = model(bow_vec)
        
        # 第四步：计算损失、梯度以及通过调用 optimizer.step() 来更新参数
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)
    
# 可以看到“creo”的权重中，与西班牙语相关的权重上升，与英语相关权重下降
print(next(model.parameters())[:, word_to_ix["creo"]])

Variable containing:
-0.9297 -0.5020
[torch.FloatTensor of size 1x2]

Variable containing:
-0.6388 -0.7506
[torch.FloatTensor of size 1x2]

Variable containing:
-0.1488
-0.1313
[torch.FloatTensor of size 2]

Variable containing:
-0.2093 -1.6669
[torch.FloatTensor of size 1x2]

Variable containing:
-2.5330 -0.0828
[torch.FloatTensor of size 1x2]

Variable containing:
 0.2803
-0.5605
[torch.FloatTensor of size 2]



可以看到我们得到了正确的结果。对于第一个样本，西班牙语对应的对数概率更高，第二个样本英语对应的对数概率更高。这与测试数据集是一致的。

现在我们看到了如何定制PyTorch组件、传递数据和梯度更新。我们为深入理解深度NLP做好了准备。